In [ ]:
# Z-score transformation of fluorescence traces in circular arena data
# Z-score is calculated based on median and median absolute deviation (MAD)
# Saves Z-score transformed data to a new CSV file

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os

# Define the file path as a string
filename = "path/to/your/data.csv"  # Replace with your actual file path

# Ensure the file exists and handle errors
try:
    # Attempt to read the CSV file into a DataFrame
    data = pd.read_csv(filename)
except FileNotFoundError:
    print(f"Error: File {filename} not found.")
    exit()
except pd.errors.EmptyDataError:
    print(f"Error: File {filename} is empty.")
    exit()
except pd.errors.ParserError:
    print(f"Error: File {filename} is not in a valid CSV format.")
    exit()

# Display the first few rows to verify the data
print("First few rows of the data:")
print(data.head())

# Identify cell columns (columns starting with 'cell-')
cell_columns = [col for col in data.columns if col.startswith('cell-')]
print("Cell columns identified:", cell_columns)

# Extract time and fluorescence data
time = data.iloc[:, 0]  # First column is assumed to be time
F = data[cell_columns]  # Fluorescence traces for each cell

# Function to calculate median-based Z-score
def median_zscore(F_cell):
    """
    Calculate the median-based Z-score for a given fluorescence trace.
    
    Parameters:
    F_cell (pd.Series or np.array): Fluorescence trace for a single cell.
    
    Returns:
    np.array: Z-score transformed fluorescence trace.
    """
    median_F = np.median(F_cell)  # Calculate the median of the fluorescence trace
    mad_F = np.median(np.abs(F_cell - median_F))  # Calculate the median absolute deviation (MAD)
    
    if mad_F == 0:
        # If MAD is zero, return zeros to avoid division by zero
        return np.zeros_like(F_cell)
    
    # Calculate and return the Z-score
    return (F_cell - median_F) / mad_F

# Function to plot the Z-score signal
def plot_zscore_signal(time, z_score, cell_name):
    """
    Plot the Z-score signal for a given cell.
    
    Parameters:
    time (pd.Series): Time values for the x-axis.
    z_score (np.array): Z-score values for the y-axis.
    cell_name (str): Name of the cell for the plot title.
    """
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=time,
        y=z_score,
        mode='lines',
        name=f'Z-score ({cell_name})',
        line=dict(color='red', width=1)
    ))
    
    # Define the exact time for the vertical line, which is the moment of entry into a new compartment zone.
    specific_time = 100  # seconds
    
    # Add a vertical red dotted line at the specific time
    fig.add_shape(
        type="line",
        x0=specific_time, x1=specific_time,
        y0=min(z_score), y1=max(z_score),
        line=dict(color="red", width=2, dash="dot")
    )
    
    # Update layout for better visualization
    fig.update_layout(
        title=f"Z-score Signal for {cell_name}",
        xaxis_title='Time (s)',
        yaxis_title='Median-based Z-score',
        legend_title='Signal Type'
    )
    fig.show()

# Dictionary to store Z-score transformed data
Z_dict = {}

# Loop through each cell trace
for cell in cell_columns:
    F_cell = F[cell]  # Extract fluorescence trace for the current cell
    Z = median_zscore(F_cell)  # Calculate Z-score for the current cell
    
    # Plot Z-score signal for the current cell
    plot_zscore_signal(time, Z, cell)
    
    # Store Z-score values in the dictionary
    Z_dict[cell] = Z

# Create a DataFrame from the Z-score dictionary
df_final = pd.DataFrame(Z_dict)
df_final.insert(0, 'time', time)  # Insert the time column at the beginning

# Define output file path
output_dir = "path/to/output/directory"  # Replace with your actual output directory
output_filename = "filename_zscore.csv"  # Define the output filename
output_filepath = os.path.join(output_dir, output_filename)

# Save the Z-score transformed data to a CSV file
df_final.to_csv(output_filepath, index=False)
print(f"Z-score data saved to {output_filepath}")